The code below will take as input a SVS image, and the user can provide a crop from this image so we can get:
* The original High-Resolution (HR) crop
* Bicubic upscaling of the Low-Resolution (LR) version of the crop
* Super-Resolution upscaling of the LR version of the crop from two different trained versions of the EDSR x4 model
* Corresponding values of MSE, PSNR and SSIM respect the upscalings and the original HR crop.

The user must have beforehand:
1. A properly set EDSR-PyTorch repository.
2. The main_use.py file we made, and properly locate it in the src folder of the EDSR-PyTorch repository.
3. The SVS image to use (and provide the path ot its file).
4. The state dictionary files of the trained versions of the EDSRx4 model (and provide the path to the corresponding files).
5. Fill the corresponding variables in the code below when needed.

We will use the directories:

In [1]:
import os

dir_base = os.getcwd()
dir_edsrpytorch = os.path.join(dir_base,"EDSR-PyTorch")     # ./EDSR-PyTorch
dir_src = os.path.join(dir_edsrpytorch,"src")               # ./EDSR-PyTorch/src
dir_pretrain = os.path.join(dir_edsrpytorch, "pre-train")   # ./EDSR-PyTorch/pre-train
dir_images = os.path.join(dir_base,"images")                # ./images

We will also require to use the OpenSlide library, so the user must have it installed before. We can do it in Google Colab with the code:

In [2]:
try:
  import openslide
except:
  !apt update && apt install -y openslide-tools
  !pip install openslide-python
  import openslide

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Fetched 2,351 kB in 4s (575 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be up

# 1. Getting the repository

In [3]:
# Cloning the repository as instructed in https://github.com/sanghyun-son/EDSR-PyTorch
!git clone https://github.com/thstkdgus35/EDSR-PyTorch {dir_edsrpytorch}

Cloning into '/content/EDSR-PyTorch'...
remote: Enumerating objects: 806, done.
remote: Total 806 (delta 0), reused 0 (delta 0), pack-reused 806
Receiving objects: 100% (806/806), 63.09 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (516/516), done.


# 2. Get the main_use.py file

The user must download it beforehand and place it inside of the src folder of the EDSR-PyTorch repository.

# 3. Get the SVS image

The user must download it beforehand and place it inside of the dir_images folder.

# 4. Get the pretrained models

The user must download them beforehand and place them inside of the dir_pretrain folder. We can also get the pretrained EDSR x4 model from its authors with the code below.

In [4]:
import os
import urllib.request

# Pre-train folder
if not os.path.exists(dir_pretrain):
    os.makedirs(dir_pretrain, exist_ok=True)

# Pretrained model
pretrain_model = "edsr_x4-4f62e9ef.pt"
pretrain_model_path = os.path.join(dir_pretrain, pretrain_model)

# Download it if not present
if not os.path.isfile(pretrain_model_path):
  url = "https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt"
  with urllib.request.urlopen(url) as response, open(pretrain_model_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)
  print(f"Pretrained model {pretrain_model} has been downloaded inside {dir_pretrain}")
else :
  print(f"Using pretrained model {pretrain_model_path}")

Pretrained model edsr_x4-4f62e9ef.pt has been downloaded inside /content/EDSR-PyTorch/pre-train


# 5. Displaying of the SVS image

The final image will be saved as comp_name in the folder provided in path_to_save.

## 5.1 Functions definitions

In [8]:
from PIL import Image, ImageDraw, ImageFont
from skimage.metrics import structural_similarity as skimage_ssim
import cv2
import math
import numpy as np
import os

## Returns a list of paths for all the files with a certain extension inside a directory and its subdirectories
def get_filepaths(directory, file_extension=""):
    files_list = []
    # Walk through the directory and its subdirectories
    for dir, subdirs, files in os.walk(directory):
        for file in files:
            # Check if the file has a the extension
            if file.endswith(file_extension):
                # Add the full path to the list
                files_list.append(os.path.join(dir, file))

    return files_list


## Function to rescale image with text to fit into a vertical space.
def fit_image_to_space(image, vertical_space, image_text="",
                       font=ImageFont.load_default()):
  # Determine text height
  draw_text = ImageDraw.Draw(image)
  text_height = draw_text.textbbox((0, 0), image_text, font)[3]

  # Determine scale factor
  image_space = vertical_space - text_height
  image_width = image.size[0]
  image_height = image.size[1]
  scale_factor = image_space / image_height

  # Rescale image
  new_image_width = math.floor(image_width * scale_factor)
  new_image_height = math.floor(image_height * scale_factor)
  new_image = image.resize((new_image_width, new_image_height), Image.BICUBIC)

  return new_image


## Function to make a composition of images as described below.
def create_composition(base_image, export_path, images=[], images_text=[],
                       hist_image=None, rescale_hist=True,
                       font=ImageFont.load_default(), font_color="black"):
  """
  Will take a base image and place it to the left on a composition. Up to 4
  other images will be displayed to the right, on a 2x2 fashion (left to right,
  top to bottom), with their corresponding text below each. An histogram can
  be also displayed to the most right of the composition.

  Will save the final composition image in "export_path", which is the absolute
  path to the filename of the desired image, including its file extension.

  All images must be PIL images.

  images and images_text must be each a list of the same length, of maximum length
  equal to 4. Each image on "images" must be of the same dimensions. Each text on
  "images_text" must have the same number of lines.
  """

  # Get the size of the base image
  base_size = base_image.size

  # Get the size of the histogram image
  if hist_image :
    # Rescale histogram if desired
    if rescale_hist :
      hist_size = hist_image.size
      new_hist_height = base_size[1]
      new_hist_width = math.floor(hist_size[0] * base_size[1] / hist_size[1])
      hist_image = hist_image.resize((new_hist_width, new_hist_height), Image.BICUBIC)

    hist_size = hist_image.size
  else :
    hist_size = (0,0)

  # Get canvas height
  canvas_height = max(base_size[1], hist_size[1])

  # If there are internal images, get their size
  if images :
    # Make a list for the new rescaled images
    new_images = []
    # Rescale internal images
    for index, image in enumerate(images) :
      new_images.append(fit_image_to_space(image=image,
                                           vertical_space = canvas_height // 2,
                                           image_text=images_text[index],
                                           font=font) )
    # Get sizes
    images_size = new_images[0].size
  else:
    images_size = (0,0)

  # Get canvas width
  canvas_width = base_size[0] + 2*images_size[0] + hist_size[0]  # without margins
  # Define internal margins
  margins_proportions = 0.01
  margin_size = math.ceil(margins_proportions * canvas_width)
  # Get canvas width with margins
  if images: canvas_width += 2*margin_size
  if hist_image: canvas_width += margin_size

  # Create a new image for the composition
  composition_size = (canvas_width, canvas_height)
  composition = Image.new("RGB", composition_size, "white")

  # Define coordinates to paste the images
  current_horizontal = 0
  current_vertical = 0

  # Paste base image
  composition.paste(base_image, (current_horizontal, current_vertical))

  # Paste internal images and write their text, if any
  if images:
    current_horizontal = base_size[0] + margin_size
    current_vertical = 0
      # Define the internal positions
    internal_positions = [
        [current_horizontal, current_vertical],
        [current_horizontal + images_size[0] + margin_size, current_vertical],
        [current_horizontal, current_vertical + canvas_height // 2],
        [current_horizontal + images_size[0] + margin_size, current_vertical + canvas_height // 2]
    ]
      # Write draw text object
    draw_text = ImageDraw.Draw(composition)
    for index in range(len(images)):
      # Paste image
      composition.paste(new_images[index], (internal_positions[index][0], internal_positions[index][1]))
      # Write text
      text_width = draw_text.textbbox((0, 0), images_text[index], font)[2]
      text_position = (internal_positions[index][0] + (images_size[0] - text_width) // 2,
                      internal_positions[index][1] + images_size[1])
      draw_text.text(text_position, images_text[index], fill=font_color, font=font, align="center")

  # Paste histogram image, if any
  if hist_image :
    current_horizontal = base_size[0] + margin_size
    if images: current_horizontal += 2*(images_size[0]+margin_size)
    current_vertical = 0

    composition.paste(hist_image, (current_horizontal, current_vertical))

  # Save the final composition image
  composition.save(export_path)


## Getting left image for the composition
def get_base_image(slide, width=1000, level=""):
  # Define level
  if not level:
    level = slide.level_count - 1
  # Define location and size
  location = (0,0)
  size = slide.level_dimensions[level]
  # Read image
  image = slide.read_region(location, level, size)
  # Rescale image to desired width
  height = width * size[1] // size[0]
  image = image.resize((width, height), Image.BICUBIC)

  return image

## Getting crop of original image for the composition
def get_original_crop(slide, start_relative=(0.0,0.0), crop_size=(1,1), level=0,
                      DesiredMagnification=20.0):
  # Get rescaling factor depending on Magnification
  ApparentMagnification = int(slide.properties["aperio.AppMag"])
  MagnificationRatio = max(ApparentMagnification/DesiredMagnification,1.0)    #It's 1 in case the Desired one is bigger than the maximum available

  # Get crop dimensions
  crop_width = math.floor(crop_size[0] * MagnificationRatio)
  crop_height = math.floor(crop_size[1] * MagnificationRatio)

  # Get crop starting position (relative must be from 0 to 1)
  image_size = slide.level_dimensions[level]
  start_x = math.floor(image_size[0] * start_relative[0])
  start_y = math.floor(image_size[1] * start_relative[1])
  start_point = (start_x, start_y)

  # Read image
  image = slide.read_region(start_point, level, (crop_width, crop_height))

  # Rescale image to desired size if needed
  if MagnificationRatio > 1.0 :
    image = image.resize(crop_size, Image.BICUBIC)
    new_width = image_size[0] // MagnificationRatio
    new_height = image_size[1] // MagnificationRatio
    image_size = (new_width,new_height)

  return image, image_size


## Bicubic upscaling of a PIL image
def get_bicubic_image(image, scale=1):
  width, height = image.size
  width = width * scale
  height = height * scale
  new_image = image.resize((width, height), Image.BICUBIC)
  return new_image


## Convert PIL image to array (by saving and loading)
def image_to_array(image, temp_folder="", keep_on_disk=False):
  """
  Function to start with a PIL Image and get its equivalent array.

  This reason of making this function like this, is that when doing it with:

    image_array = np.array(image_PIL)

  We were getting discrepancies of values while measuring PSNR. I.e., an
  image_array like above would get a PSNR different from the image_array
  defined below. The latter is then preferrable.
  """
  # Get tem folder
  if not temp_folder :
    temp_folder = os.getcwd()
  # Save temp image
  image_name = "temp_image_to_array.png"
  file_path = os.path.join(temp_folder, image_name)
  image.save(file_path)

  # Load temp image as array
  image_array = cv2.imread(file_path)

  # Delete temp image
  if not keep_on_disk :
    try:
      # Attempt to remove the file
      os.remove(file_path)
    except OSError as e:
      # Handle errors, if any
      print(f"Error: {e}")

  return image_array

## Convert PIL image to array (by saving and loading)
def array_to_image(image_array, temp_folder="", keep_on_disk=False):
  """
  Function to start with an array and get its equivalent PIL Image,
  by saving it in disk and loading it.
  """
  # Get tem folder
  if not temp_folder :
    temp_folder = os.getcwd()
  # Save temp array
  image_array_name = "temp_array_to_image.png"
  file_path = os.path.join(temp_folder, image_array_name)
  cv2.imwrite(file_path, image_array)

  # Load temp image as array
  image = Image.open(file_path)

  # Delete temp image
  if not keep_on_disk :
    try:
      # Attempt to remove the file
      os.remove(file_path)
    except OSError as e:
      # Handle errors, if any
      print(f"Error: {e}")

  return image


## Mean Squared Error between two images
def calc_mse(imageA, imageB):
  """
  'Mean Squared Error' between two images of same dimensions and type (RGB, for example)

  Images must be an array (e.g. Array of iunt8 of shape (height,width,3))

  MSE = sum over N of (Ai - Bi)^2 / N     where N is the number of pixels per image

  Images are converted to float32 precision.
  """
  imageA_float32 = imageA.astype(np.float32)
  imageB_float32 = imageB.astype(np.float32)

  MSE = np.sum((imageA_float32 - imageB_float32)**2)
  MSE /= math.prod(imageA_float32.shape[dim] for dim in range(len(imageA_float32.shape)))

  return MSE


## Peak-Signal-To-Noise Ratio between a SR and a HR image
def calc_psnr_simplified(sr, hr, scale, rgb_range=255, crop=True):
  """
  Notes:
    * sr and hr must be a numpy array.
    * sr and hr are converted to float32 precision.
    * sr and hr might be cropped from their borders with a number "shave" of
    pixels, if crop=True.
  """
  diff = (sr.astype(np.float32) - hr.astype(np.float32))  # Ensure float32 precision
  if crop :
    shave = scale + 6
    valid = diff[shave:-shave, shave:-shave, ...]         # Cropping out borders of the image
  else :
    valid = diff
  mse = np.mean(valid ** 2)
  psnr = -10 * math.log10(mse / (rgb_range ** 2))

  return psnr

## Structural Similarity between two images
def calc_ssim(imageA, imageB, channel_axis=2):
  """
  'Structural Similarity' between two images, calculated with "structural_similarity"
  from the library skimage.metrics.

  Images must be an array (e.g. Array of iunt8 of shape (height,width,3)) and
  are converted to float32 precision.

  The argument channel_axis correspond to the index of the arrays corresponding
  to their color channels (typically 0 or 2).
  """
  imageA_float32 = imageA.astype(np.float32)
  imageB_float32 = imageB.astype(np.float32)

  SSIM = skimage_ssim(imageA_float32, imageB_float32, channel_axis=channel_axis)

  return SSIM

## Determine the MSE, PSNR OR SSIM of a list of images respect an original image
def make_measurements(image_original, images=[], measure="", temp_folder=""):
  # All images must be a numpy array of same shape (height, width, channels)
  if temp_folder == "":
    temp_folder = os.getcwd()
  img_original = image_to_array(image_original, temp_folder=temp_folder, keep_on_disk=False)

  # Make measurements
  measurements = []
  for image in images :
    # Convert image to numpy array
    img = image_to_array(image, temp_folder=temp_folder, keep_on_disk=False)
    if measure == "MSE" :
      #Mean Squared Error
      try:
        MSE = calc_mse(img_original, img)
      except ValueError:
        MSE = ""
      measurements.append(MSE)
    elif measure == "PSNR":
      #Peak Signal-to-Noise Ratio
      try:
        PSNR = calc_psnr_simplified(img, img_original, scale, rgb_range=255, crop=True)
      except ValueError:
        PSNR = ""
      measurements.append(PSNR)
    elif measure == "SSIM":
      #Structural Similarity
      try:
        SSIM = calc_ssim(img_original, img, channel_axis=2)
      except ValueError:
        SSIM = ""
      measurements.append(SSIM)
    else :
      measurements.append("N/A")

  return measurements

## Round values on a list or assign "N/A" if not possible
def round_values_in_list(list_values, decimals=2, na_value="N/A"):
  new_list = []
  for value in list_values :
    try :
      val = f"{round(value,decimals):.{decimals}f}"
    except:
      val = na_value
    new_list.append(val)

  return new_list


## Draw a rectangle inside an image (NOTE: this will modify the original image)
def draw_rectangle(image, start_relative=(0.0,0.0), dimensions=(1,1),
                   frame_color="red", frame_width=1, original_size=""):
  # Get width,height dimensions
  if original_size :
    width = math.floor(dimensions[0] * image.size[0] / original_size[0])
    height = math.floor(dimensions[1] * image.size[1] / original_size[1])
  else :
    width = dimensions[0]
    height = dimensions[1]
  # Get starting point (relative must be from 0 to 1)
  start_x = math.floor(image.size[0] * start_relative[0])
  start_y = math.floor(image.size[1] * start_relative[1])

  # Create a frame inside the image
  draw = ImageDraw.Draw(image)
  draw.rectangle([(start_x,start_y), (start_x + width, start_y + height)],
                  fill=None, outline=frame_color, width=frame_width)
  return

In [9]:
import os
from PIL import Image
import shutil

## Apply EDSR model to all JPG/PNG images inside folder
#NOTE: Make sure images are RGB and not RGBA
def apply_EDSR_to_folder(folder="./EDSR-PyTorch/test",
                         data_test="Demo", scale=4, pretrain_path="",
                         dir_src="./EDSR-PyTorch/src", verbose=True):
  ## Change the current working directory to EDSR-PyTorch/src, to run the model
  starting_dir = os.getcwd()
  current_dir = starting_dir
  # Check if our directory is ./EDSR-PyTorch/src
  if current_dir.endswith("EDSR-PyTorch/src"):
    if verbose :
      print(f"[INFO] The Current working directory is: {current_dir}\n")
  else:
    # Changing the current working directory
    os.chdir(dir_src)
    if verbose:
      print(f"[INFO] The Current working directory is: {os.getcwd()}\n")

  ## Use the model
  if verbose :
    print(f"[INFO] Using EDSR_x{scale} model as {data_test} with weights from {pretrain_path} on images inside {folder}.\n")
  !python main_use.py --data_test {data_test} --dir_demo {folder} --scale {scale} --save test --n_resblocks 32 --n_feats 256 --res_scale 0.1 --pre_train {pretrain_path} --test_only --save_results

  # Change the current working directory back to the previous one
  os.chdir(starting_dir)
  if verbose :
    print(f"[INFO] The Current working directory is: {starting_dir}\n")

  return


## Apply EDSR to a single image (returned image is an array)
def apply_EDSR_to_image(image, folder="./EDSR-PyTorch/test",
                        data_test="Demo", scale=4, pretrain_path="",
                        dir_src="./EDSR-PyTorch/src", verbose=True,
                        keep_on_disk=False):
  # Make copy of image for manipulation (without modifying the original one)
  input_image = image.copy()

  # If image not RGB, make it RGB
  if input_image.mode != "RGB":
    input_image = input_image.convert("RGB")

  # Save image on disk
  if not os.path.exists(folder):
    os.makedirs(folder, exist_ok=True)
  image_path = os.path.join(folder,"current_image.png")
  input_image.save(image_path)

  # Use EDSR model
  apply_EDSR_to_folder(folder=folder, data_test=data_test, scale=scale,
                       pretrain_path=pretrain_path, dir_src=dir_src, verbose=verbose)

  # Read upscaled image
  upscaled_image_filename = f"current_image_x{scale}.png"
  upscaled_image_path = os.path.join(folder,"results",upscaled_image_filename)
  upscaled_image = cv2.imread(upscaled_image_path)

  if not keep_on_disk :
    # Delete created directories and files
    try:
      # Attempt to remove the directory and its contents
      shutil.rmtree(folder)
    except OSError as e:
      # Handle errors, if any
      print(f"Error: {e}")

  # Return upscaled image
  return upscaled_image

## 5.2 Variables

In [25]:
import os

# Required directories
current_image_folder = os.path.join(dir_images, "current_image")            # ./images/current_image
temp_image_folder = os.path.join(dir_images, "temp_image")                  # ./images/temp_image

for folder_path in [current_image_folder, temp_image_folder] :
  if not os.path.exists(folder_path):
      os.makedirs(folder_path, exist_ok=True)


# SVS image
svs_image_names = ["C3N-00167-21.svs"]                # Could be a single name or a list of names
svs_image_list = [os.path.join(dir_images, image_name) for image_name in svs_image_names]
#svs_image_list = get_filepaths(dir_images, file_extension=".svs")      # This will retrieve the path to all .svs files in dir_images


# Model variables
DesiredMagnification = 20.0
scale = 4
# EDSR 1
model1_name = "EDSR pretrained"
model1_filename = "edsr_x4-4f62e9ef.pt"
pretrain_path1 = os.path.join(dir_pretrain, model1_filename)
# EDSR 2
model2_name = "EDSR trained"
model2_filename = "edsr_x4-best_2024-03-19.pt"
pretrain_path2 = os.path.join(dir_pretrain, model2_filename)


# Measures
measures = ["MSE", "PSNR", "SSIM"]


# Image composition variables
comp_name = "pretrained_vs_trained"
path_to_save = dir_images

base_image_width = 1000
start_relative = (0.588,0.267)          # Pixels (34000, 12000) respects image dimension (57767, 44968)
crop_size = (500,500)


# Aesthetic variables
frame_color = "red"
frame_width = 10
font=ImageFont.truetype(font="/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf",size=15)
font_color="black"


## 5.3 Displaying image

In [26]:
import openslide

current_iteration = 1

# Go over each SVS image
for svs_image_path in svs_image_list :
  # Image name
  image_name, _ = os.path.splitext(os.path.basename(svs_image_path))

  number_of_iterations = len(svs_image_list)
  print(f"Processing image {image_name}: {current_iteration}/{number_of_iterations}")

  # Load SVS image as slide
  slide = openslide.OpenSlide(svs_image_path)

  # Get base image
  base_image = get_base_image(slide, width=base_image_width)

  # Get 4 inner images
    #Original
  image_original, full_image_size = get_original_crop(slide, start_relative=start_relative,
                                                      crop_size=crop_size, level=0,
                                                      DesiredMagnification=DesiredMagnification)
    # Make sure the original one is RGB
  if image_original.mode != "RGB":
    image_original = image_original.convert("RGB")

    #Downscale it
  image_original_array = image_to_array(image_original, temp_folder=temp_image_folder, keep_on_disk=False)
  image_downscaled_array = cv2.resize(image_original_array, None, fx=1/scale, fy=1/scale, interpolation= cv2.INTER_AREA)
  image_downscaled = array_to_image(image_downscaled_array, temp_folder=temp_image_folder, keep_on_disk=False)

    #Bicubic
  image_bicubic = get_bicubic_image(image_downscaled, scale=scale)
    #EDSR 1
  image_edsr1_array = apply_EDSR_to_image(image_downscaled, folder=current_image_folder,
                                          data_test="Demo", scale=scale, pretrain_path=pretrain_path1,
                                          dir_src=dir_src, verbose=False)
  image_edsr1 = array_to_image(image_edsr1_array, temp_folder=temp_image_folder, keep_on_disk=False)
    #EDSR 2
  image_edsr2_array = apply_EDSR_to_image(image_downscaled, folder=current_image_folder,
                                          data_test="Demo", scale=scale, pretrain_path=pretrain_path2,
                                          dir_src=dir_src, verbose=False)
  image_edsr2 = array_to_image(image_edsr2_array, temp_folder=temp_image_folder, keep_on_disk=False)

  inner_images = [image_original, image_bicubic, image_edsr1, image_edsr2]
  # Get measurements for 4 inner images
  if len(measures) == 1:
    measure=measures[0]
    measurements = make_measurements(image_original=image_original, images=inner_images,
                                     measure=measure, temp_folder=temp_image_folder)
    measurements = round_values_in_list(measurements,decimals=2, na_value="")

    inner_images_text = [f"Original",
                        f"Bicubic\n{measure}: {measurements[1]}",
                        f"{model1_name}\n{measure}: {measurements[2]}",
                        f"{model2_name}\n{measure}: {measurements[3]}"]
  else :
    inner_images_text = [f"Original\n | ",
                        f"Bicubic\n",
                        f"{model1_name}\n",
                        f"{model2_name}\n"]

    for measure_idx, measure in enumerate(measures):
      measurements = make_measurements(image_original=image_original, images=inner_images,
                                       measure=measure, temp_folder=temp_image_folder)
      measurements = round_values_in_list(measurements,decimals=2, na_value="")

      # Add measurements
      for text_idx in range(1,4):
        if measure_idx > 0:
          inner_images_text[text_idx] = f"{inner_images_text[text_idx]} | "
        inner_images_text[text_idx] = f"{inner_images_text[text_idx]}{measure}: {measurements[text_idx]}"

  # Draw square on base image
  draw_rectangle(base_image, start_relative=start_relative, dimensions=crop_size,
                 frame_color=frame_color, frame_width=frame_width,
                 original_size=full_image_size)

  # Make composition and save
  export_folder = path_to_save
  if not os.path.exists(export_folder):
    os.makedirs(export_folder, exist_ok=True)

  composition_name = f"{comp_name}.png"

  export_path = os.path.join(export_folder,composition_name)

  create_composition(base_image=base_image, export_path=export_path,
                     images=inner_images, images_text=inner_images_text,
                     font=font, font_color=font_color)

  print(f"Composition {composition_name} was successfully saved in {export_folder}.\n")

  # End
  current_iteration += 1

Processing image C3N-00167-21: 1/1
Placing images inside /content/images/current_image into dataloader...
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Loading model...
Making model...
Load the model from /content/EDSR-PyTorch/pre-train/edsr_x4-4f62e9ef.pt
Processing images...
100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.38s/it]
All images were successfully processed.
Placing images inside /content/images/current_image into dataloader...
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoa